Part 1: Data Ingestion

In [7]:
import requests
try:
    local_filename = "yellow_taxi_data.parquet"
    url ="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print("yelllow completed")         
except Exception as e:
    print("\n the following error occured with downloading the parquet file: ", e)

try:    
    local_filename2 = "taxi_zone_data.csv"
    url2 ="https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
    with requests.get(url2, stream=True) as r:
        r.raise_for_status()
        with open(local_filename2, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print("zone completed") 
except Exception as e:
    print("\n the following error occured with downloading the parquet file: ", e)


    



 the following error occured with downloading the parquet file:  403 Client Error: Forbidden for url: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet

 the following error occured with downloading the parquet file:  403 Client Error: Forbidden for url: https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv


Part 1.2 
    Data Validation

In [5]:
import polars as pl
required_col = ["tpep_pickup_datetime","tpep_dropoff_datetime","PULocationID","DOLocationID","passenger_count","trip_distance","fare_amount","fare_amount","total_amount","payment_type"]
datetime_field =["tpep_pickup_datetime","tpep_dropoff_datetime"]
try:
    file = "yellow_taxi_data.parquet"
    df= pl.read_parquet(file)

    #compares columns in df to columns in required col
    contains_col =all(column in df.columns for column in required_col)# true for all the condition( if column in the dataset is in the required field list) 
    if contains_col:
        print ("dataset has all required columns")
    else:
        missing_col =all(column not in df.columns for column in required_col)#true for all the condition(if column in the dataset is not in the required field)
        print ("dataset has is missing required columns",missing_col)

    datetime_col= df.select(datetime_field).schema
    #print(datetime_col)
    if all(datetime_col[column] == pl.Datetime for column in datetime_field):#true for all condition(if column in the datetime_list ,is in the dataset and is of type datetime )
         print ("dataset has all required datetime columns")
    else:
         print ("dataset MISSING required datetime columns")

    initial_row=df.height
    print("processing parquet file ")
    print(f'Number of rows:',df.height )
    print(f'Number of columns: {len(df.columns)}')
    print(df.schema)
except Exception as e:
    print("\n the following error occured with the parquet file: ", e)

try:
    print("\nprocessing csv file ")
    file2 = ("taxi_zone_data.csv")
    df2 = pl.read_csv(file2)
    print(f'Number of rows:',df2.height )
    print(f'Number of columns: {len(df2.columns)}')
    print(df2.schema)
except Exception as e:
    print("the following error occured with the csv file: ", e)



 the following error occured with the parquet file:  No such file or directory (os error 2): yellow_taxi_data.parquet

processing csv file 
the following error occured with the csv file:  No such file or directory (os error 2): taxi_zone_data.csv


"Part 2: Data Transformation & Analysis


In [6]:
#filter out null values
df=df.drop_nulls(pl.col("tpep_pickup_datetime"))
df=df.drop_nulls(pl.col("tpep_dropoff_datetime"))
df=df.drop_nulls(pl.col("PULocationID"))
df=df.drop_nulls(pl.col("DOLocationID"))
df=df.drop_nulls(pl.col("fare_amount"))


#fiilter out invalid and outlier values
df=df.filter(pl.col("trip_distance") > 0)
print(df)
df=df.filter(pl.col("fare_amount") > 0)
df=df.filter(pl.col("fare_amount") < 500)
df=df.filter(pl.col("tpep_dropoff_datetime") > pl.col("tpep_pickup_datetime"))

#number of rows removed
removed_rows = initial_row - (df.height)
print(f'Number of removed rows:', removed_rows, " these row were removed becaused they had no analytical value such as null values or inconsistent/il-logical data such as negative trip distance and pickup times eariler than dropoff times ")

NameError: name 'df' is not defined

Part 2.2 feature engineering 


In [ ]:
#feature engineering 
#feature trip_duration_minutes
df=(df.with_columns((pl.col("tpep_dropoff_datetime")- pl.col("tpep_pickup_datetime")).alias("trip_duration_minutes")))
#feature trip_speed_mph
df=df.with_columns(df.with_columns((pl.col("trip_distance").truediv(pl.col("trip_duration_minutes").cast(pl.Float64))).alias("trip_speed_mph")))
#feature pickup_hour
df=df.with_columns(pl.col("tpep_pickup_datetime").dt.hour().alias("pickup_hour"))
#feature pickup_day_of_week
df=df.with_columns(pl.col("tpep_pickup_datetime").dt.strftime('%A').alias("pickup_day_of_week"))

creating a sample for trip distance 
    this was done due to rounding the trip distance values for the histogram
    the variance for population and smaple was taken to ensure spread
    

In [ ]:
#getting varicence of population trip distance and a sample of 100000 varicence of sample trip distance
print(df.select(pl.var("trip_distance")))
df_sample = df.select("trip_distance").sample(n=100000, seed=42)
print(df_sample.select(pl.var("trip_distance")))

SQL Analysis

Part 2.3.1 What are the top 10 busiest pickup zones by total number of trips? (Include zone names from lookup table)

In [ ]:

#What are the top 10 busiest pickup zones by total number of trips? (Include zone names from lookup table)
import duckdb as db

db = db.connect()

result = db.execute(''' 
                    SELECT c.Zone AS pickup_name,
                    COUNT(*) as total_trips
                    FROM 'yellow_taxi_data.parquet' as p
                    JOIN 'taxi_zone_data.csv' as c
                    ON p.PULocationID = c.LocationID
                    GROUP BY c.Zone
                    ORDER BY total_trips DESC
                    LIMIT 10
            ''').fetchdf()

print(result)


this query answers which places are most frequent by taxi

Part 2.3.2 What is the average fare amount for each hour of the day? (Order by hour)

In [ ]:
#What is the average fare amount for each hour of the day? (Order by hour)
result2 = db.execute('''
                     SELECT 
                     EXTRACT(HOUR FROM tpep_pickup_datetime) AS hour,
                     AVG(fare_amount) AS avg_fare
                     FROM 'yellow_taxi_data.parquet' 
                     GROUP BY hour
                     ORDER BY hour ASC
                     ''').fetchdf()
print(result2)

this query answer the question: which hour had the highest earning during the day , this can be affected by long trips compared to other hours or many short trips ?


part 2.3.3 What percentage of trips use each payment type?


In [ ]:
#What percentage of trips use each payment type?
result3 = db.execute('''
                     SELECT payment_type,
                     count(*) * 100 / sum(count(*)) OVER() as percentage,
                     FROM 'yellow_taxi_data.parquet'
                     GROUP BY payment_type
                     ORDER BY percentage DESC
                     ''').fetchdf()

print(result3)
                     

this answers which payment type is the most popular

part 2.3.4 What is the average tip percentage (tip_amount/fare_amount) by day of week, for
credit card payments only?

In [ ]:
#pay type 1= card
#What is the average tip percentage (tip_amount/fare_amount) by day of week, for credit card payments only?
result4 = db.execute('''
                     SELECT EXTRACT( WEEKDAY FROM tpep_pickup_datetime ) AS day_week,
                     AVG(tip_amount / fare_amount)*100 AS avg_tip_percent,
                     FROM 'yellow_taxi_data.parquet'
                     WHERE payment_type = 1
                     AND fare_amount > 0
                     GROUP BY day_week 
                     ORDER BY day_week ASC
                     ''').fetchdf()


print(result4)


this query answer the question for credit card payments only, which day do driver get a higher tip percentage ?

part 2.3.5  What are the top 5 most common pickup-dropoff zone pairs? (Include zone names)


In [ ]:
#What are the top 5 most common pickup-dropoff zone pairs? (Include zone names)
result5 = db.execute('''
    SELECT
        pu.Zone AS pickup_zone,
        dz.Zone AS dropoff_zone,
        COUNT(*) AS total_trips
    FROM 'yellow_taxi_data.parquet' AS p
    JOIN 'taxi_zone_data.csv' AS pu
        ON p.PULocationID = pu.LocationID
    JOIN 'taxi_zone_data.csv' AS dz
        ON p.DOLocationID = dz.LocationID
    GROUP BY pu.Zone, dz.Zone
    ORDER BY total_trips DESC
    LIMIT 5
''').fetchdf()

print(result5)

#gpt


this query answer the question where do most people travel to and from?

Part 3 visualization prototyping

bar chart Top 10 pickup zones by trip count

In [ ]:

import matplotlib.pyplot as plt
from matplotlib.patches import Patch


#Bar chart: Top 10 pickup zones by trip count, result 1

num_trips=result["total_trips"]


colors = [
    "#2FA82B" if t > 140000 else
    "#2C3E50" if 100000 <= t < 140000 else
    "#E74C3C" 
    for t in (num_trips)
]


plt.figure(figsize=(28,15))
plt.rcParams['axes.axisbelow'] = True
plt.grid()
bars =plt.bar(result["pickup_name"],result["total_trips"],color=colors,edgecolor='white')
plt.xlabel('Pickup Zones', fontsize=12)
plt.ylabel('Number of Trips', fontsize=12)
plt.title('NYC Taxi trips by location', fontsize=14)
plt.xticks(range(10))


legend_elements = [
    Patch(facecolor='#2FA82B', label='trips greater than 140000'),
    Patch(facecolor='#2C3E50', label='trips between 100000 and 140000 '),
    Patch(facecolor='#E74C3C', label='trips less than 100000')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()

#plt.bar(range(len(y)), y, width=0.3, align='center', color='skyblue', zorder=3)


Line chart: Average fare by hour of day (showing hourly patterns)

In [ ]:
# Line chart: Average fare by hour of day (showing hourly patterns)
import plotly.express as px
fig = px.line(result2,x="hour",y="avg_fare",title="New York taxi average fare",labels={'hour': 'Hour', 'avg_fare': 'Average fares'})

fig.update_traces(mode='lines', hovertemplate='%{x}<br>Trips: %{y:,}')
fig.update_layout(height=500, hovermode='x unified')
fig.update_xaxes(
    tickmode='linear',
    tickvals=list(range(23))
)
fig.show()  

Histogram: Distribution of trip distances (with appropriate binning) for sample and populations data


In [ ]:
#Histogram: Distribution of trip distances (with appropriate binning)
import plotly.express as px

df_sample=df_sample.filter(pl.col("trip_distance") > 0)
df_sample=df_sample.filter(pl.col("trip_distance") < 50)
df_sample=df_sample.with_columns(pl.all().round(mode="half_away_from_zero"))
print(df_sample.select(pl.var("trip_distance")))

#fig = px.histogram(df_sample, x="trip_distance", nbins=50, title="Distribution of Trip Distances")
fig.show()


trip_distance_pop=df.select(pl.col("trip_distance"))
trip_distance_pop=trip_distance_pop.filter(pl.col("trip_distance") > 0)
trip_distance_pop=trip_distance_pop.filter(pl.col("trip_distance") < 50)

print("variance before rounding popluation ",trip_distance_pop.select(pl.var("trip_distance")))

trip_distance_pop=trip_distance_pop.with_columns(pl.all().round(mode="half_away_from_zero"))
print("variance after rounding population",trip_distance_pop.select(pl.var("trip_distance")))

figg = px.histogram(trip_distance_pop, x="trip_distance", nbins=50, title="Distribution of Trip Distances")
figg.show()
#a sample was taken at first to local machine processing limitations but after some changes , was able to process the populaton



bar chart showing Breakdown of payment types

In [ ]:
#bar chart: Breakdown of payment types
pay_type=result3["payment_type"]

colors = [
    "#2FA82B" if p == 0 else
    "#2C3E50" if p==1 else
    "#791193" if p==4 else
    "#E74C3C" if p ==2 else
    "#A8AE0F" 
    for p in (pay_type)
]


plt.figure(figsize=(28,15))
plt.rcParams['axes.axisbelow'] = True
plt.grid()
fig=plt.bar(result3["payment_type"],result3["percentage"],color=colors,edgecolor='white')
plt.xlabel('Paymet Type', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.title('NYC Taxi trips payment types', fontsize=14)

legend_elements = [
    Patch(facecolor='#2FA82B', label='cash'),
    Patch(facecolor='#2C3E50', label='credit card'),
    Patch(facecolor='#E74C3C', label='no charge'),
    Patch(facecolor='#A8AE0F', label='dispute'),
    Patch(facecolor='#791193', label='unknown')
]

plt.legend(handles=legend_elements, loc='upper right')

plt.figure(figsize=(12,8))

#fig =px.bar(result3, x="payment_type", y="percentage")

chart heatmap showing trips by day of week and hour (showing weekly patterns)

In [ ]:
#trips by day of week and hour (showing weekly patterns)
import pandas as pd
import numpy as np
import plotly.express as px
#heatmap
pdf = df["pickup_day_of_week","pickup_hour"].to_pandas()

order_week = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ordered = pd.CategoricalDtype(categories=order_week,ordered=True)
pdf["pickup_day_of_week"] = pdf["pickup_day_of_week"].astype(ordered)

grouped_by_day_hour=pdf.groupby(["pickup_day_of_week","pickup_hour"]).value_counts()
trip_counts=grouped_by_day_hour.reset_index(name="trip_count")

heatmap_format=trip_counts.pivot(index="pickup_day_of_week",columns="pickup_hour",values="trip_count")

fig=px.imshow(heatmap_format)
fig.show()
